<h1>Part 4 - Experiment Tracking</h1>

# Experiment Tracking and Model Management with MLFlow

There are many ways to use the MLFlow Tracking API. For simple local uses, the best is to leave the data management to MLFlow and let it store runs, metrics, models and artifacts locally. For more advanced usage, all of this information can be stored in databases. You can find the detailed on MLFlow's documentation [here](https://mlflow.org/docs/latest/tracking.html#scenario-1-mlflow-on-localhost).

## Exploring MLFlow

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

Let's print the tracking server URI, where the experiments and runs are going to be logged. We observe it refers to a local path.

In [1]:
import mlflow

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///d:/Git/stephina/lessons/01-model-and-experiment-management/mlruns'


After this initialization, we can connect create a client to connect to the API and see what experiments are present.

By refering to mlflow's [documentation](https://mlflow.org/docs/latest/python_api/mlflow.client.html), create a client and display a list of the available experiments using the search_experiments function. This function could prove useful later to programatically explore experiments (rather than in the UI)

In [1]:
from mlflow.tracking import MlflowClient

# Create an instance of the MlflowClient
mlflow_client = MlflowClient()

We see that there is a default experiment for which the runs are stored locally in the mlruns folder.

### Creating an experiment and logging a new run

An experiment is a logical entity regrouping the logs of multiple attempts at solving a same problem, called runs. \
We will now work with the classic sklearn dataset iris. Our goal here is to manage to classify the different iris species. To track our models performance, we will log every attempt as a "run" and create a new experiment "iris-experiment-1" to regroup them.

Lookup the mlflow.run and mlflow.start_run functions [here](https://mlflow.org/docs/latest/python_api/mlflow.html?highlight=start_run#mlflow.start_run) to find out how to manage runs.
Explore [this part](https://mlflow.org/docs/latest/python_api/mlflow.html) to learn more about the log_params, log_metrics and log_artifact functions. Find out how to log sklearn models [here](https://mlflow.org/docs/latest/python_api/mlflow.sklearn.html])

Complete the following in order to log the parameters, interesting metrics and the model.

In [2]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Set the MLflow experiment
mlflow.set_experiment("iris-experiment-1")

# Start MLflow run
with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Load Iris dataset
    X, y = load_iris(return_X_y=True)

    # Define model parameters
    params = {"C": 0.1, "random_state": 42}

    # Train the Logistic Regression model
    model = LogisticRegression(**params).fit(X, y)

    # Make predictions
    y_pred = model.predict(X)

    # Calculate accuracy
    accuracy = accuracy_score(y, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)

    # Log parameters
    mlflow.log_params(params)

    # Save the model as an MLflow artifact
    mlflow.sklearn.log_model(model, "model")

    # Print the default artifacts URI
    print(f"Default artifacts URI: '{mlflow.get_artifact_uri()}'")


2024/01/10 20:00:48 INFO mlflow.tracking.fluent: Experiment with name 'iris-experiment-1' does not exist. Creating a new experiment.


Default artifacts URI: 'file:///d:/Git/stephina/lessons/01-model-and-experiment-management/mlruns/751613127420444897/6504cbb5d57c44a494d7803411987768/artifacts'


d:\Anaconda\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
experiments = mlflow.search_experiments()
experiments

[<Experiment: artifact_location='file:///d:/Git/stephina/lessons/01-model-and-experiment-management/mlruns/751613127420444897', creation_time=1704913248921, experiment_id='751613127420444897', last_update_time=1704913248921, lifecycle_stage='active', name='iris-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///d:/Git/stephina/lessons/01-model-and-experiment-management/mlruns/0', creation_time=1704913205700, experiment_id='0', last_update_time=1704913205700, lifecycle_stage='active', name='Default', tags={}>]

Try running the training script with various parameters to have runs to compare.
You can now explore your run(s) using the ui: \
(Paste "mlflow ui --host 0.0.0.0 --port 5002" in your terminal, or run the cell below)

**N.B.** Make sure you are in the lecture folder and not the repo root!

In [5]:
!mlflow ui --host 0.0.0.0 --port 5002

'mlflow' is not recognized as an internal or external command,
operable program or batch file.


You will have to kill the cell to continue experimenting

### Interacting with the model registry

If you are satisfied with the last run's model, you can transform the logged model into a registered model. It will be logged in the Model Registry, which makes it easier to use in production and manage versions.

In [7]:
# We already have our run id from above. Let's use it to register the model

result = mlflow.register_model(f"runs:/{run_id}/models", "iris_lr_model")

Registered model 'iris_lr_model' already exists. Creating a new version of this model...
Created version '2' of model 'iris_lr_model'.


# Use Case

Now we will get back to our taxi rides use case: 

In [8]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from typing import List
from scipy.sparse import csr_matrix

## 0 - Download Data

In [9]:
!pip install gdown

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     -------------------------------------- 57.6/57.6 kB 764.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
import gdown
import os

DATA_FOLDER = "../../data"
train_path = f"{DATA_FOLDER}/yellow_tripdata_2021-01.parquet"
test_path = f"{DATA_FOLDER}/yellow_tripdata_2021-02.parquet"
predict_path = f"{DATA_FOLDER}/yellow_tripdata_2021-03.parquet"


if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)
    print(f"New directory {DATA_FOLDER} created!")

gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet",
    train_path,
    quiet=False,
)
gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet",
    test_path,
    quiet=False,
)
gdown.download(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet",
    predict_path,
    quiet=False,
)

Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet
To: d:\Git\stephina\data\yellow_tripdata_2021-01.parquet
100%|██████████| 21.7M/21.7M [00:01<00:00, 18.6MB/s]
Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet
To: d:\Git\stephina\data\yellow_tripdata_2021-02.parquet
100%|██████████| 21.8M/21.8M [00:01<00:00, 15.5MB/s]
Downloading...
From: https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet
To: d:\Git\stephina\data\yellow_tripdata_2021-03.parquet
100%|██████████| 30.0M/30.0M [00:01<00:00, 17.2MB/s]


'../../data/yellow_tripdata_2021-03.parquet'

## 1 - Load data

In [11]:
def load_data(path: str):
    return pd.read_parquet(path)


train_df = load_data(train_path)
train_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN


## 2 - Prepare the data

Let's prepare the data to make it Machine Learning ready. \
For this, we need to clean it, compute the target (what we want to predict), and compute some features to help the model understand the data better.

### 2-1 Compute the target

We want to predict a taxi trip duration in minutes. Let's compute it as a difference between the drop-off time and the pick-up time for each trip.

In [12]:
def compute_target(
    df: pd.DataFrame,
    pickup_column: str = "tpep_pickup_datetime",
    dropoff_column: str = "tpep_dropoff_datetime",
) -> pd.DataFrame:
    df["duration"] = df[dropoff_column] - df[pickup_column]
    df["duration"] = df["duration"].dt.total_seconds() / 60
    return df


train_df = compute_target(train_df)

In [13]:
train_df["duration"].describe()

count    1.369769e+06
mean     1.391168e+01
std      1.312006e+02
min     -1.350846e+05
25%      5.566667e+00
50%      9.066667e+00
75%      1.461667e+01
max      2.881770e+04
Name: duration, dtype: float64

Let's remove outliers and reduce the scope to trips between 1 minute and 1 hour

In [14]:
MIN_DURATION = 1
MAX_DURATION = 60


def filter_outliers(df: pd.DataFrame, min_duration: int = 1, max_duration: int = 60) -> pd.DataFrame:
    return df[df["duration"].between(min_duration, max_duration)]


train_df = filter_outliers(train_df)

### 2-2 Prepare features

#### 2-2-1 Categorical features

Most machine learning models don't work with categorical features. Because of this, they must be transformed so that the ML model can consume them.

In [15]:
CATEGORICAL_COLS = ["PUlocationID", "DOlocationID"]


def encode_categorical_cols(df: pd.DataFrame, categorical_cols: List[str] = None) -> pd.DataFrame:
    if categorical_cols is None:
        categorical_cols = ["PULocationID", "DOLocationID", "passenger_count"]
    df[categorical_cols] = df[categorical_cols].fillna(-1).astype("int")
    df[categorical_cols] = df[categorical_cols].astype("str")
    return df


train_df = encode_categorical_cols(train_df)

In [17]:
def extract_x_y(
    df: pd.DataFrame,
    categorical_cols: List[str] = None,
    dv: DictVectorizer = None,
    with_target: bool = True,
) -> dict:

    if categorical_cols is None:
        categorical_cols = ["PULocationID", "DOLocationID", "passenger_count"]
    dicts = df[categorical_cols].to_dict(orient="records")

    y = None
    if with_target:
        if dv is None:
            dv = DictVectorizer()
            dv.fit(dicts)
        y = df["duration"].values

    x = dv.transform(dicts)
    return x, y, dv


X_train, y_train, dv = extract_x_y(train_df)

## 3 - Train model

We train a basic linear regression model to have a baseline performance

In [18]:
def train_model(x_train: csr_matrix, y_train: np.ndarray):
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    return lr


model = train_model(X_train, y_train)

## 4 - Evaluate model

We evaluate the model on train and test data

### 4-1 On train data

In [19]:
def predict_duration(input_data: csr_matrix, model: LinearRegression):
    return model.predict(input_data)


def evaluate_model(y_true: np.ndarray, y_pred: np.ndarray):
    return mean_squared_error(y_true, y_pred, squared=False)


prediction = predict_duration(X_train, model)
train_me = evaluate_model(y_train, prediction)
train_me

6.782411957484398

### 4-2 On test data

In [22]:
test_df = load_data(test_path)

In [23]:
test_df = compute_target(test_df)
test_df = encode_categorical_cols(test_df)
X_test, y_test, _ = extract_x_y(test_df, dv=dv)

In [24]:
y_pred_test = predict_duration(X_test, model)
test_me = evaluate_model(y_test, y_pred_test)
test_me

58.37505345540477

## 5 - Log Model Parameters to MlFlow

Now that all our development functions are built and tested, let's create a training pipeline and log the training parameters, logs and model to MlFlow.

Create a training flow, log all the important parameters, metrics and model. Try to find what could be important and needs to be logged.

In [25]:
experiment_name = "stephina_experiment"

# Check if the experiment exists
existing_experiment = mlflow.get_experiment_by_name(experiment_name)

# If the experiment doesn't exist, create a new one
if existing_experiment is None:
    mlflow.create_experiment(experiment_name)
    print(f"Experiment '{experiment_name}' created.")
else:
    print(f"Using existing experiment '{experiment_name}'.")


Experiment 'stephina_experiment' created.


In [26]:
# Set a name and description for your model
model_name = "taxi_duration_model_v1"
model_description = "Linear Regression Model for Taxi Duration Prediction"

# Set a description for your model
model.description = model_description

# Log your model
mlflow.sklearn.log_model(model, "model")

# Register your model in MLflow Model Registry
mlflow.register_model("runs:/{}/model".format(run_id), model_name)


Successfully registered model 'taxi_duration_model_v1'.
Created version '1' of model 'taxi_duration_model_v1'.


<ModelVersion: aliases=[], creation_timestamp=1704913555890, current_stage='None', description=None, last_updated_timestamp=1704913555890, name='taxi_duration_model_v1', run_id='6504cbb5d57c44a494d7803411987768', run_link=None, source='file:///d:/Git/stephina/lessons/01-model-and-experiment-management/mlruns/751613127420444897/6504cbb5d57c44a494d7803411987768/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [27]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from typing import List
from scipy.sparse import csr_matrix

# Set the experiment name
mlflow.set_experiment("stephina_experiment")

# Check if there's an active run, and end it if necessary
if mlflow.active_run():
    mlflow.end_run()

# Start a run
with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Set tags for the run
    mlflow.set_tag("experiment_id", run_id)

    # Load data
    train_df = load_data(train_path)
    test_df = load_data(test_path)

    # Compute target
    train_df = compute_target(train_df)
    test_df = compute_target(test_df)

    # Filter outliers
    train_df = filter_outliers(train_df)
    test_df = filter_outliers(test_df)

    # Encode categorical columns
    train_df = encode_categorical_cols(train_df)
    test_df = encode_categorical_cols(test_df)

    # Extract X and y for training set
    X_train, y_train, dv = extract_x_y(train_df)

    # Train model
    model = train_model(X_train, y_train)

    # Evaluate model on the training set
    train_predictions = predict_duration(X_train, model)
    train_rmse = evaluate_model(y_train, train_predictions)
    mlflow.log_metric("train_rmse", train_rmse)

    # Extract X and y for test set
    X_test, y_test, _ = extract_x_y(test_df, dv=dv)

    # Evaluate model on the test set
    test_predictions = predict_duration(X_test, model)
    test_rmse = evaluate_model(y_test, test_predictions)
    mlflow.log_metric("test_rmse", test_rmse)

    # Log your model
    mlflow.sklearn.log_model(model, "model")

    # Register your model in MLflow Model Registry
    model_name = "taxi_duration_model_v1"
    model_description = "Linear Regression Model for Taxi Duration Prediction"
    mlflow.register_model("runs:/{}/model".format(run_id), model_name)


Registered model 'taxi_duration_model_v1' already exists. Creating a new version of this model...
Created version '2' of model 'taxi_duration_model_v1'.


If the model is satisfactory, we stage it as production using the appropriate version. This will help us retreiving it for predictions.

Create a mlflow client and use the [mlflow documentation](https://mlflow.org/docs/latest/python_api/mlflow.client.html?highlight=transition_model_version_stage#mlflow.client.MlflowClient.transition_model_version_stage) to stage the appropriate model as being in "production".

In [ ]:
client = MlflowClient()
...

## 6 - Predict

We can now use our model to predict on fresh unseen data and forecast what is going to be the duration of a tawi trip depending on trip characteristics.

In [ ]:
# Load prediction data
predict_df = load_data(predict_path)

# Apply feature engineering
predict_df = encode_categorical_cols(predict_df)
X_pred, _, _ = extract_x_y(predict_df, dv=dv, with_target=False)

# Load production model
model_uri = f"models:/{mlflow_experiment_path}/production"
model = mlflow.sklearn.load_model(model_uri)

# Make predictions
y_pred = predict_duration(X_pred, model)
y_pred

## 7 - To go further

If you managed to go this far, you can try solving the use case using an other regression model like [XGBoost](https://xgboost.readthedocs.io/en/stable/) for instance.